In [1]:
!git clone https://github.com/JHA-Lab/cnn_design-space.git
%cd cnn_design-space/visualization
%ls

Cloning into 'cnn_design-space'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 153 (delta 35), reused 58 (delta 20), pack-reused 72
Receiving objects: 100% (153/153), 74.53 MiB | 33.37 MiB/s, done.
Resolving deltas: 100% (45/45), done.
/content/cnn_design-space/visualization
cnnbench_results.ipynb  nasbench_var.ipynb


In [2]:
import json
import os
import re
import tensorflow as tf
import copy
import numpy as np
import random
import sys

if '../' not in sys.path:
    sys.path.append('../')
    
from cnnbench.lib import config as _config

# Set DEBUG to True to print intermediate variables
DEBUG = True

# Define model directory
model_dir = '../results/vertices_2/'

In [4]:
config = _config.build_config()
dataset = None

with open(os.path.join(model_dir, 'generated_graphs.json')) as f:
    models = json.load(f)

if DEBUG: print(f'All hashes: {models.keys()}')    

All hashes: dict_keys(['043721b9c7fe8c5fad811d47d83132ec'])


In [5]:
loaded_dataset = tf.data.TFRecordDataset(os.path.join(model_dir, 'cnnbench.tfrecord'))

features = {
        'module_adjacency': tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True),
        'module_operations': tf.io.FixedLenFeature([], tf.string),
        'trainable_parameters': tf.io.FixedLenFeature([], tf.int64),
        'training_time': tf.io.FixedLenFeature([], tf.float32),
        'train_accuracy': tf.io.FixedLenFeature([], tf.float32),
        'validation_accuracy': tf.io.FixedLenFeature([], tf.float32),
        'test_accuracy': tf.io.FixedLenFeature([], tf.float32)}

def _parse_image_function(example_proto):
  return tf.io.parse_single_example(example_proto, features)

parsed_dataset = loaded_dataset.map(_parse_image_function)
parsed_dataset
for elem in parsed_dataset:
    raw_adjacency = elem['module_adjacency'].numpy()
    dim = int(np.sqrt(len(raw_adjacency)))
    adjacency = np.array([int(e) for e in list(raw_adjacency)], dtype=np.int8)
    adjacency = np.reshape(adjacency, (dim, dim))
    
    operations = elem['module_operations'].numpy().decode("utf-8")
    trainable_parameters = elem['trainable_parameters'].numpy()
    training_time = elem['training_time'].numpy()
    train_accuracy = elem['train_accuracy'].numpy()
    validation_accuracy = elem['validation_accuracy'].numpy()
    test_accuracy = elem['test_accuracy'].numpy()
    print(f'Adjacency matrix: \n{adjacency} \nOperations: {operations} \nTrainable parameters: {trainable_parameters}')
    print(f'Train Accuracy: {train_accuracy} \nValidation Accuracy: {validation_accuracy} \nTest Accuracy: {test_accuracy}\n')

Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: input,output 
Trainable parameters: 882570
Train Accuracy: 0.19412946701049805 
Validation Accuracy: 0.19161184132099152 
Test Accuracy: 0.19701522588729858

Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: input,output 
Trainable parameters: 882570
Train Accuracy: 0.1274999976158142 
Validation Accuracy: 0.1171875 
Test Accuracy: 0.13181090354919434

Adjacency matrix: 
[[0 1]
 [0 0]] 
Operations: input,output 
Trainable parameters: 882570
Train Accuracy: 0.10830356925725937 
Validation Accuracy: 0.1003289446234703 
Test Accuracy: 0.10637019574642181

